## Тестирование Mongodb

In [38]:
import os

os.system('docker run -d -p 27017:27017 --name mongo mongo:latest')

2846c433d3ec4f8482eddc270118b778434348252fbdf3b8006e2ec601cdb215


0

### Подключаемся к БД

In [39]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [40]:
from pymongo import MongoClient, ASCENDING
import uuid
import random
from datetime import date

client = MongoClient('localhost', 27017)
db = client.test_database
rating_collection = db.rating
review_collection = db.review

## Тесты

In [41]:
data = []
movie_uuids = [str(uuid.uuid4()) for _ in range(1000)]
user_uuids = [str(uuid.uuid4()) for _ in range(1000)]
for _ in range(1_000_000):
    test_data = {
        "movie_uuid": random.choice(movie_uuids),
        "user_uuid": random.choice(user_uuids),
        "rating": random.randint(0, 10)
    }
    data.append(test_data)
rating_collection.insert_many(data)
rating_collection.create_index([('movie_uuid', ASCENDING)])

'movie_uuid_1'

In [42]:
review_data = []
review_rating_data = []
for _ in range(100_000):
    id_ = str(uuid.uuid4())
    test_review_data = {
        "_id": id_,
        "text": "cool film",
        "first_name": "Damon",
        "last_name": "Smith",
        "date": date.today().isoformat(),
        "movie_uuid": random.choice(movie_uuids),
        "user_uuid": random.choice(user_uuids),
        "rating": random.randint(0, 10)
    }
    review_data.append(test_review_data)
    for _ in range(10):
        test_rating_data = {
            "review_id": id_,
            "user_uuid": random.choice(user_uuids),
            "rating": random.choice([0, 10])
        }
        review_rating_data.append(test_rating_data)

review_collection.insert_many(review_data)
rating_collection.insert_many(review_rating_data)
review_collection.create_index([('movie_uuid', ASCENDING)])
rating_collection.create_index([('review_id', ASCENDING)])

In [49]:
%%time
pipeline = [
    {
        "$match": {"movie_uuid": movie_uuids[0]}
    },
    {
        "$project": {
            "movie_uuid": 1,
            "rating": 1,
            "equal0": {
                "$cond": [{"$eq": ["$rating", 0]}, 1, 0]
            },
            "equal10": {
                "$cond": [{"$eq": ["$rating", 10]}, 1, 0]
            }
        }
    },
    {
        "$group": {
            "_id": "$movie_uuid",
            "like": {"$sum": "$equal10"},
            "dislike": {"$sum": "$equal0"},
            "rating": {"$avg": "$rating"}
        }
    }
]
list(rating_collection.aggregate(pipeline))

CPU times: user 1.51 ms, sys: 1.12 ms, total: 2.63 ms
Wall time: 19.9 ms


[{'_id': 'c3c08300-1ca8-4cc6-be71-294c503b8523',
  'like': 92,
  'dislike': 96,
  'rating': 4.972139303482587}]

In [56]:
%%time
review_pipeline = [
    {
        "$match": {"movie_uuid": movie_uuids[0]}
    },
    {
        "$lookup": {
            "from": "rating",
            "localField": "_id",
            "foreignField": "review_id",
            "pipeline": [
                {
                    "$project": {
                        "equal0": {
                            "$cond": [{"$eq": ["$rating", 0]}, 1, 0]
                        },
                        "equal10": {
                            "$cond": [{"$eq": ["$rating", 10]}, 1, 0]
                        }
                    }
                },
                {
                    "$group": {
                        "_id": "$movie_uuid",
                        "like": {"$sum": "$equal10"},
                        "dislike": {"$sum": "$equal0"},
                    }
                }
            ],
            "as": "ratings"
        }
    }
]
list(review_collection.aggregate(review_pipeline))

CPU times: user 1.46 ms, sys: 1.5 ms, total: 2.96 ms
Wall time: 39.9 ms


[{'_id': '90237552-8e66-4b1d-ae88-3f113b25ea6c',
  'text': 'cool film',
  'first_name': 'Damon',
  'last_name': 'Smith',
  'date': '2022-06-06',
  'movie_uuid': 'c3c08300-1ca8-4cc6-be71-294c503b8523',
  'user_uuid': '5fd66777-9bf9-46cb-839d-bd6cca75cce7',
  'rating': 5,
  'ratings': [{'_id': None, 'like': 7, 'dislike': 3}]},
 {'_id': '382ad9ac-8cb6-4c29-a7bc-a1b8d7eb6ed1',
  'text': 'cool film',
  'first_name': 'Damon',
  'last_name': 'Smith',
  'date': '2022-06-06',
  'movie_uuid': 'c3c08300-1ca8-4cc6-be71-294c503b8523',
  'user_uuid': '9e7ddc28-a2d7-482c-b501-310b1b120518',
  'rating': 5,
  'ratings': [{'_id': None, 'like': 3, 'dislike': 7}]},
 {'_id': 'cb5f41ac-0afd-4121-8a1f-1aaa3c8c1f92',
  'text': 'cool film',
  'first_name': 'Damon',
  'last_name': 'Smith',
  'date': '2022-06-06',
  'movie_uuid': 'c3c08300-1ca8-4cc6-be71-294c503b8523',
  'user_uuid': '38c44b92-00ea-479c-97b1-e4c843157be3',
  'rating': 7,
  'ratings': [{'_id': None, 'like': 3, 'dislike': 7}]},
 {'_id': 'bcae1088-

In [58]:
%%time
one_doc = {
    "movie_uuid": movie_uuids[0],
    "user_uuid": user_uuids[0],
    "rating": random.randint(0, 10)
}
rating_collection.insert_one(one_doc)

CPU times: user 1.14 ms, sys: 1.11 ms, total: 2.26 ms
Wall time: 4.57 ms
